In [1]:
import os
import pandas as pd
from dotenv import load_dotenv

if os.getcwd().split("/")[-1] == "sandbox":
    os.chdir("../")
    
    
load_dotenv()

True

In [2]:
# df_dk = pd.read_excel("food_co2_estimator/data/vector_db/DBv2.xlsx",sheet_name="DK")
# df_gb = pd.read_excel("food_co2_estimator/data/vector_db/DBv2.xlsx",sheet_name="GB")

In [3]:
# from langchain_chroma import Chroma
# from langchain_openai import OpenAIEmbeddings

# embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
# vector_store = Chroma(
#     collection_name="test_db",
#     embedding_function=embeddings,
#     persist_directory="./sandbox/test_db",  # Where to save data locally, remove if not neccesary
# )

In [4]:
# from typing import Any, Dict, List
# from uuid import uuid4

# from langchain_core.documents import Document

# emission_records_dk: List[Dict[str, Any]] = df_dk.to_dict(orient="records")
# emission_records_gb: List[Dict[str, Any]] = df_gb.to_dict(orient="records")

# documents = []
# uuids = []
# for id, (emission_record_dk, emission_record_gb) in enumerate(zip(emission_records_dk,emission_records_gb), 1):
    
#     if "Name" in emission_record_gb:   
#         documents.append(
#             Document(
#                 page_content=emission_record_gb["Name"],
#                 metadata=emission_record_dk,
#                 id=id,
#             )
#         )
#         uuids.append(str(uuid4()))
    

# vector_store.add_documents(documents)

In [5]:
from food_co2_estimator.data.vector_store import get_vector_store


vector_store = get_vector_store()

In [6]:

results = vector_store.similarity_search(query="lasagna sheets",k=5)
for res in results:
    print(f"* {res.page_content} =  {res.metadata['Total kg CO2e/kg']}")

* Lasagne, vegetarian, ready meals =  1.530188183518141
* Lasagne, beef, ready meals =  12.15720923746473
* Lasagne, vegan, ready meals =  1.45641150135913
* Pasta, raw =  2.052608339289675
* Fresh pasta, fettucine spinach =  1.526357879263283


In [7]:
retriever = vector_store.as_retriever(k=5)

retriever.batch(["tomat","kartoffel"])
    
     

[[Document(metadata={'DSK Kategori': 'Grøntsager', 'Detail': 0.00626463674608679, 'Emballage': 0.2599925713297975, 'Energi (KJ/100 g)': 1187.0, 'Fedt (g/100 g)': 3.0, 'Forarbejdning': 8.881784197001252e-16, 'ID_Ra': 'Ra00005', 'ID_food': 'Fa0152', 'ID_pack': 'Pa066', 'ID_retail': 'Ma0332', 'ILUC': 0.0854828717229545, 'Kategori': 'Grøntsager og grøntsagsprodukter', 'Kulhydrat (g/100 g)': 43.5, 'Landbrug': 2.040736124100985, 'Navn': 'Tomat, soltørret', 'Produkt': 'Tomat, soltørret', 'Protein (g/100 g)': 14.1, 'Total kg CO2e/kg': 2.719357591199155, 'Transport': 0.3268813872993297}, page_content='Tomato, dried'),
  Document(metadata={'DSK Kategori': 'Drikkevarer', 'Detail': 0.00626463674608679, 'Emballage': 0.2240555011602198, 'Energi (KJ/100 g)': 79.0, 'Fedt (g/100 g)': 0.1, 'Forarbejdning': 0.2040981936926205, 'ID_Ra': 'Ra00356', 'ID_food': 'Fa0123', 'ID_pack': 'Pa043', 'ID_retail': 'Ma0332', 'ILUC': 0.02087975329376196, 'Kategori': 'Drikkevarer', 'Kulhydrat (g/100 g)': 3.3, 'Landbrug': 

In [8]:
from typing import List
from langchain_core.runnables import RunnablePassthrough
from langchain_chroma.vectorstores import Document


def parse_retriever_output(documents: List[Document]):
    results = {}
    for document in documents:
        if "Total kg CO2e/kg" in document.metadata.keys():
            emission = document.metadata['Total kg CO2e/kg']
            emission_rounded = round(float(emission),1)
            results[document.page_content] = f"{emission_rounded} kg CO2e / kg"
    return results


retriever_chain = retriever | parse_retriever_output


def batch_retriever(inputs: List[str]):
    return dict(zip(inputs,retriever_chain.batch(inputs)))



batch_retriever(["tomat","agurk"])




{'tomat': {'Tomato, dried': '2.7 kg CO2e / kg',
  'Tomatojuice, canned': '1.1 kg CO2e / kg',
  'Tomato, peeled, canned': '1.1 kg CO2e / kg',
  'Tomato, ripe, raw, origin unknown': '0.5 kg CO2e / kg'},
 'agurk': {'Gherkin, pickled': '3.1 kg CO2e / kg',
  'Gherkin cucumber, raw': '0.4 kg CO2e / kg',
  'Cucumber, raw': '0.4 kg CO2e / kg',
  'Cucumber, large, pickled': '3.1 kg CO2e / kg'}}

In [9]:
from food_co2_estimator.chains.recipe_extractor import get_recipe_extractor_chain
from food_co2_estimator.utils.url import get_url_text, get_full_url_text

In [10]:
recipe_extractor = get_recipe_extractor_chain(verbose=False)

In [12]:
recipe_extractor

PromptTemplate(input_variables=['input'], input_types={}, partial_variables={'website_response': '\n{\n    "ingredients": [\n        "500 g torskefilet",\n        "1 tsk havsalt",\n        "2 æg",\n        "1 gulerod, fintrevet",\n        "0.5 dl fløde (13%)",\n        "0.5 tsk revet muskatnød",\n        "1 tsk peber",\n        "2 spsk olie",\n        "4 dl creme fraiche (18%)",\n        "4 æggeblommer",\n        "2 spsk frisk dild, hakket",\n        "4 spsk frisk persille, hakket"\n    ],\n    "persons": 4,\n    "instructions": "Forbered fiskefarsen ved at skære torskefileten i mindre stykker og blend den sammen med havsalt i en foodprocessor til en fin konsistens. \n    Tilsæt de to hele æg, fintrevet gulerod, fløde, muskatnød og peber. Blend igen, indtil ingredienserne er godt blandet og konsistensen er jævn. \n    Smag til med salt og peber efter behov. Forvarm ovnen til 180 grader. Smør en lille brødform eller ildfast fad med lidt olie og hæld fiskefarsen i formen. \n    Glat over

In [13]:
url = "https://www.valdemarsro.dk/frikadeller/"
# text = get_full_url_text(url)

In [14]:
text = get_url_text(url)
output = recipe_extractor.invoke(input=text)

ValidationError: 1 validation error for Generation
text
  Input should be a valid string [type=string_type, input_value=Recipe(ingredients=['500 ...il de er gennemstegte.'), input_type=Recipe]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type

In [ ]:
output

In [19]:
from typing import Any, Dict

from food_co2_estimator.output_parsers.recipe_extractor import Recipe


def parse_recipe_output(output: Recipe) -> List[str]:
    return output.ingredients

In [20]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
extractor_chain = get_url_text | recipe_extractor | parse_recipe_output

In [ ]:
extractor_chain.invoke(url)

In [28]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_openai import ChatOpenAI

from food_co2_estimator.output_parsers.sql_co2_estimator import CO2Emissions

message = """
Given this list of ingredients, return the emissions in kg / Co2e kg provided the best matching emission option. 
{ingredients}

Ingredients emission options:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0,verbose=True)

structured_llm = llm.with_structured_output(CO2Emissions)

rag_chain = {"context": batch_retriever, "ingredients": RunnablePassthrough()} | prompt | structured_llm

In [34]:
output = await rag_chain.ainvoke(["tomat","kartoffel"])

In [ ]:
output

In [ ]:
from translate import Translator
from operator import itemgetter


translator = Translator(to_lang="en", from_lang="da")

def _translate_if_danish(inputs:List[str],language: str):
    if language == "en":
        return inputs
    
    inputs_str = ", ".join(inputs)
    translations = translator.translate(inputs_str)
    return translations.split(", ")

def translate_if_danish(_dict: Dict[str,str]):
    return _translate_if_danish(inputs=_dict["inputs"],language=_dict["language"])

 
_translate_if_danish(["tomat","kartoffel"],language="da")


In [39]:
from langchain_core.runnables import RunnableLambda

translation_chain = {"inputs": itemgetter("inputs"), "language": itemgetter("language")} | RunnableLambda(translate_if_danish) | rag_chain

In [ ]:
# translation_chain = {"inputs": itemgetter("inputs"), "language": itemgetter("language")} | RunnableLambda(translate_if_danish)

translation_chain.invoke(input={"inputs":["tomat","kartoffel"],"language":"da"})

In [ ]:
output = translation_chain.invoke({"inputs":["tomat","kartoffel"],"language":"da"})

In [ ]:
print(output.emissions[0])